**LangGraph 기반 챗봇 만들기**
1. OPENAI_API_KEY를 입력받아 환경변수로 저장
2. TypedDict와 Annotated, add_messages를 사용하여 상태(State) 정의
3. StateGraph를 이용하여 챗봇 대화 흐름 구성.
그래프는 START > chatbot > END 구조
4. chatbot노드에서는 LLM을 호출하여 응답을 생성하도록 한다.
5. 사용자 입력을 받아 graph.stram()을 통해 응답을 출력하는 함수 작성.
(사용자가 "quit", "exit", "q"를 입력하면 프로그램 종료.)

In [ ]:
!pip install langchain langchain-openai openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
!pip install aiosqlite

In [ ]:
[REMOVED API KEY]

💬 챗봇과 대화를 시작하세요. (종료: 'quit', 'exit', 'q')
🙋 사용자: 배고파
🤖 챗봇: 뭘 먹고 싶나요? 저도 같이 생각해볼까요?
🙋 사용자: 음 메뉴추천해줘'
🤖 챗봇: 무슨 종류의 음식을 원하시나요? 한식, 중식, 양식, 일식 등 원하시는 음식 종류를 알려주세요. 그러면 그에 맞는 메뉴를 추천해 드릴게요.
🙋 사용자: 떡볶이
🤖 챗봇: 떡볶이는 좋은 선택이에요! 집에서 간단하게 만들어 먹을 수도 있고, 식당에서 맛있게 먹을 수도 있어요. 더 매콤하게 먹고 싶다면 고추장이나 고추가루를 추가해보세요. 떡볶이와 함께 어떤 사이드 메뉴를 먹을지도 고민해보세요. 군만두, 오뎅, 튀김 등이 잘 어울릴 거에요. 맛있게 드세요!
🙋 사용자: 사이드메뉴 뭐가 좋을까
🤖 챗봇: 떡볶이와 잘 어울리는 사이드 메뉴는 다양해요. 몇 가지 추천해드릴게요:

1. 군만두: 바삭한 피부와 부드러운 고기 속을 즐길 수 있는 군만두는 떡볶이와 찰떡궁합이에요.
2. 오뎅: 쫄깃한 식감과 바삭한 튀김옷이 맛있는 오뎅은 떡볶이와 함께 먹으면 더욱 맛있어요.
3. 튀김: 고추, 깻잎, 양파, 고구마 등 다양한 채소나 해산물을 튀긴 튀김은 떡볶이와 함께 즐기기 좋아요.

이 외에도 떡볶이와 잘 어울리는 사이드 메뉴는 다양하니, 원하시는 것을 골라서 함께 즐겨보세요!
🙋 사용자: 튀김 좋겠다!!!!!!
🤖 챗봇: 좋은 선택이에요! 튀김을 곁들여 떡볶이를 먹으면 더욱 풍부한 맛을 느낄 수 있어요. 바삭하고 고소한 튀김이 떡볶이의 매콤하고 달콤한 맛과 잘 어울리죠. 고추, 깻잎, 양파 등 다양한 튀김 종류를 준비해서 떡볶이와 함께 즐겨보세요. 맛있는 식사 되세요!
🙋 사용자: q
👋 챗봇을 종료합니다.


전체 흐름 요약
```
사용자 입력 → 상태에 저장 → LangGraph 실행 → GPT 응답 생성 → 상태 업데이트 → 반복


In [ ]:
[REMOVED API KEY]

💬 챗봇과 대화를 시작하세요.
   ↪ 종료: 'quit', 'exit', 'q'
   ↪ 대화 요약: '/요약'
🙋 사용자: 안녕
🤖 챗봇: 안녕하세요! 무엇을 도와드릴까요?
🙋 사용자: 오늘 점심 추천해줘
🤖 챗봇: 오늘은 어떤 음식이 땡기시나요? 한식, 중식, 양식 등 원하시는 음식 종류를 알려주시면 그에 맞는 추천을 해드릴게요.
🙋 사용자: 한식
🤖 챗봇: 한식으로는 불고기나 된장찌개, 불닭 등이 맛있는 선택이 될 수 있어요. 불고기는 달콤한 고기와 식감이 좋은 채소가 함께 익혀진 요리로, 된장찌개는 간장과 된장으로 만들어진 국물 요리입니다. 불닭은 매운 양념을 고기에 씌워 구워낸 요리로, 매운 음식을 좋아하시면 좋을 것 같아요.어떠세요?
🙋 사용자: 불고기 맛있겠네
🤖 챗봇: 불고기는 직접 만드실 수도 있고, 외식을 하실 때 주문하실 수도 있어요. 만약 직접 만들기를 원하신다면 재료를 준비하시고 레시피를 찾아보시면 도움이 될 거예요. 외식을 하실 경우에는 한식당이나 한식뷔페에서 불고기를 주문하셔서 맛있게 즐기시면 되겠네요. 맛있는 식사 하시길 바라며, 좋은 점심 되세요!
🙋 사용자: /요약
📝 요약: 챗봇이 한식 중에서 불고기를 추천해주고, 직접 만들거나 외식할 수 있는 방법을 안내해주면서 맛있는 식사를 기원하는 대화였습니다.
🙋 사용자: q
👋 챗봇을 종료합니다.
